In [ ]:
# Extracting features from well known public repos for training.

import glob
import os

training_data_files_path = '/home/kc/Projects/data_files/Training_data_from_public_git/'                     

git_repo_folder = '/home/kc/Projects/git-server/'
public_git_repos = [file for file in os.listdir(git_repo_folder) if file.endswith(".git")]

#print(len(public_git_repos))
# Split it into 4 sublists - we will run 4 notebooks to speed up the process
public_git_repos_1 = public_git_repos[0:6]
public_git_repos_2 = public_git_repos[7:14]
public_git_repos_3 = public_git_repos[15:21]
public_git_repos_4 = public_git_repos[22:28]
public_git_repos_5 = public_git_repos[29:35]
public_git_repos_6 = public_git_repos[36:42]
public_git_repos_7 = public_git_repos[43:44]

In [ ]:
for repo in public_git_repos_2:
    if DEBUG >= 1:
        print('Starting mining for: ', repo)
    p1 = create_components('git://localhost/'+repo,'',localdir)
    p1_commits = get_latest_commits(p1[0],p1[1],p1[2])
    p1_commits.to_csv(training_data_files_path+repo+'.csv')

In [ ]:
kmeans = KMeans(n_clusters = 5, init='k-means++',max_iter = 20)
s = kmeans.fit(data_scaled)
s_centroids = s.cluster_centers_

plt.scatter(data_scaled[:,0], data_scaled[:,1], s=50, alpha=0.5)
plt.scatter(s_centroids[:, 0], s_centroids[:,1], c='red', s=50)
plt.show()
# atulya test

In [ ]:
# Dimentionality Reduction before training with Kmeans.
# Dimentionality Reduction is the technique used to identify the most influential features for machine learning
# The input data in this case contains just ten features. Hence, this may not be required
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA

# Assigning the input frame to a simpler variable
df = ml_commits_numeric_na
# Creating the instance of Standard Scaler
scaler = StandardScaler()
# Transforming the input data using Standard Scaler
data_scaled = scaler.fit_transform(df)
# Creating the instance of PCA. PCA is a popular tool for the purpose of Dimentionality Reduction
pca = PCA()
# Processing the scaled data using PCA
pca.fit(data_scaled)
# Printing the variance ratio for each feature. FYI, each column of the frame represents a feature
pca.explained_variance_ratio_

In [ ]:
#Plotting the variance ratios for each feature
import matplotlib.pyplot as plt
plt.figure(figsize = (12,8))
plt.plot(range(1,7),pca.explained_variance_ratio_.cumsum(),marker = 'o',linestyle = '--')
plt.title('Explained Variance by Component')
plt.xlabel('Number of Components')
plt.ylabel('Cumulatve Explained Variance')

In [ ]:
# From the above plot, it is clear that first three features have major impact on variance. And the last five have no imapct at all.
# Hence, we can try Kmeans with first three features
pca = PCA(n_components = 6)
pca.fit(data_scaled)
scores_pca = pca.transform(data_scaled)
print(scores_pca.shape)

In [ ]:
# Training the input frame with PCA processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
df = ml_commits_nout_numeric
scaler = StandardScaler()
normalizer = Normalizer()
data_scaled = scaler.fit_transform(df)


pca = PCA(n_components = 6)
pca.fit(data_scaled)
scores_pca = pca.transform(data_scaled)
kmeans = KMeans(n_clusters = 5, init='random',max_iter = 10)
q = kmeans.fit(scores_pca)

plt.scatter(scores_pca[:,0], scores_pca[:,1], s=50, alpha=0.5)
plt.scatter(q_centroids[:, 0], q_centroids[:, 1], c='red', s=50)
plt.show()

In [ ]:
# Function to try out various input models
# Includes measurement of quality of Clustering through silhouette_scores
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

print(__doc__)

# X can be assigned to any input frame.
X = data_normalized
# If you want to try out Scaled frame, replace the above line with the following
# X = data_scaled
# For PCA frame, replace with the following line
# X = scores_pca
range_n_clusters = [2, 3, 4, 5, 6,8]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10,max_iter = 50,init = 'random')
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values =  sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    # In the below line, two features, X[:,4] and X[:,6] are selected for plotting
    # Those features can be replaced with any other two features.
    # Keep in mind that, the frame contains ten features, from X[:,0] to X[:,9]
    # In the case of PCA, there are only three features selected after the dimentionality reduction.
    # Hence, if the X is assigned to scores_pca, you can select from X[:,0] to X[:2]
    ax2.scatter(X[:, 4], X[:, 6], marker='.', s=150, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    # The centers should have same positions as features. 
    ax2.scatter(centers[:,4], centers[:,6], marker='o',
                c="red", alpha=1, s=300, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[4], c[6], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()

In [ ]:
# Function to try out various input models
# This is same as the above except that the input is limited to just two features
# The puprose here is to understand how cluster behaves if we have same number of training features as plotted
# Includes measurement of quality of Clustering through silhouette_scores
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

print(__doc__)

# X can be assigned to any input frame.
X = data_normalized[:,[3,5]]
# If you want to try out Scaled frame, replace the above line with the following
# X = data_scaled[:,[3,5]]
# For PCA frame, replace with the following line
# X = scores_pca[:,[0,1]]
range_n_clusters = [2, 3, 4, 5, 6,8]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10,max_iter = 50,init = 'random')
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values =  sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    # Here, we can only plot X[:, 0], X[:, 1] as the training was done on just two features
    ax2.scatter(X[:, 0], X[:, 1], marker='.', s=150, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    # The centers should have same positions as features. 
    ax2.scatter(centers[:,0], centers[:,1], marker='o',
                c="red", alpha=1, s=300, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()

In [ ]:
import scipy.stats as ss
data_scaled = scaler.fit_transform(ml_commits_all_na)
num_cols = len(scores_pca[0,:])
for i in range(num_cols):
    col = scores_pca[:,i]
    col_stats = ss.describe(col)
    print(col_stats)

In [ ]:
import elasticsearch
from elasticsearch_dsl import Search,Q
import pandas as pd
#p1[0],p1[1],p1[2]
es = p1[0]
es_ma_index = p1[1]
es_bl_index =p1[2]
# Using Elasticsearch DSL function to get the data of Commit index
#print(es,es_ma_index,es_bl_index)
blame_es_data = Search(using=es, index=es_bl_index)
        # Loading data into a dictionary
blame_dict = [hit.to_dict() for hit in blame_es_data.scan()]
print(len(blame_dict))
        # Using Elasticsearch DSL function to get the data of Blame index
commit_es_data = Search(using=es, index=es_ma_index)
        # Loading data into a dictionary
commit_dict = [hit.to_dict() for hit in commit_es_data.scan()]
print(len(commit_dict))
        # Creating pandas dataframe for commit data
commit_frame = pd.DataFrame(commit_dict)
        # Creating pandas dataframe for blame data
blame_frame = pd.DataFrame(blame_dict)
        # Getting the blame row count. If the frmae is empty, it means all the records are clean
blame_count = blame_frame.shape[0]

In [ ]:
# https://gist.github.com/shreyasbapat/89c6d6e09ff3f763e21ea68f98d74f84
# This script outputs relevant system environment info
# Run it with `python collect_env.py`.
from __future__ import absolute_import, division, print_function, unicode_literals
import re
import subprocess
import sys
import time
import datetime
import os
from collections import namedtuple

try:
    import torch
    TORCH_AVAILABLE = True
except (ImportError, NameError, AttributeError):
    TORCH_AVAILABLE = False

PY3 = sys.version_info >= (3, 0)

# System Environment Information
SystemEnv = namedtuple('SystemEnv', [
    'torch_version',
    'is_debug_build',
    'cuda_compiled_version',
    'gcc_version',
    'cmake_version',
    'os',
    'python_version',
    'is_cuda_available',
    'cuda_runtime_version',
    'nvidia_driver_version',
    'nvidia_gpu_models',
    'cudnn_version',
    'pip_version',  # 'pip' or 'pip3'
    'pip_packages',
    'conda_packages',
])


def run(command):
    """Returns (return-code, stdout, stderr)"""
    p = subprocess.Popen(command, stdout=subprocess.PIPE,
                         stderr=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    rc = p.returncode
    if PY3:
        output = output.decode("utf-8")
        err = err.decode("utf-8")
    return rc, output.strip(), err.strip()


def run_and_read_all(run_lambda, command):
    """Runs command using run_lambda; reads and returns entire output if rc is 0"""
    rc, out, _ = run_lambda(command)
    if rc is not 0:
        return None
    return out


def run_and_parse_first_match(run_lambda, command, regex):
    """Runs command using run_lambda, returns the first regex match if it exists"""
    rc, out, _ = run_lambda(command)
    if rc is not 0:
        return None
    match = re.search(regex, out)
    if match is None:
        return None
    return match.group(1)


def get_conda_packages(run_lambda):
    if get_platform() == 'win32':
        grep_cmd = r'findstr /R "torch soumith mkl magma"'
    else:
        grep_cmd = r'grep "torch\|soumith\|mkl\|magma"'
    out = run_and_read_all(run_lambda, 'conda list | ' + grep_cmd)
    if out is None:
        return out
    # Comment starting at beginning of line
    comment_regex = re.compile(r'^#.*\n')
    return re.sub(comment_regex, '', out)


def get_gcc_version(run_lambda):
    return run_and_parse_first_match(run_lambda, 'gcc --version', r'gcc (.*)')


def get_cmake_version(run_lambda):
    return run_and_parse_first_match(run_lambda, 'cmake --version', r'cmake (.*)')


def get_nvidia_driver_version(run_lambda):
    smi = get_nvidia_smi()
    return run_and_parse_first_match(run_lambda, smi, r'Driver Version: (.*?) ')


def get_gpu_info(run_lambda):
    smi = get_nvidia_smi()
    uuid_regex = re.compile(r' \(UUID: .+?\)')
    rc, out, _ = run_lambda(smi + ' -L')
    if rc is not 0:
        return None
    # Anonymize GPUs by removing their UUID
    return re.sub(uuid_regex, '', out)


def get_running_cuda_version(run_lambda):
    return run_and_parse_first_match(run_lambda, 'nvcc --version', r'V(.*)$')


def get_cudnn_version(run_lambda):
    """This will return a list of libcudnn.so; it's hard to tell which one is being used"""
    if get_platform() == 'win32':
        cudnn_cmd = 'where /R "%CUDA_PATH%\\bin" cudnn*.dll'
    else:
        cudnn_cmd = 'find /usr/local /usr/lib -type f -name "libcudnn*" 2> /dev/null'
    rc, out, _ = run_lambda(cudnn_cmd)
    # find will return 1 if there are permission errors or if not found
    if len(out) == 0:
        return None
    if rc != 1 and rc != 0:
        return None
    # Alphabetize the result because the order is non-deterministic otherwise
    result = '\n'.join(sorted(out.split('\n')))
    return 'Probably one of the following:\n{}'.format(result)


def get_nvidia_smi():
    smi = 'nvidia-smi'
    if get_platform() == 'win32':
        smi = '"C:\\Program Files\\NVIDIA Corporation\\NVSMI\\%s"' % smi
    return smi


def get_platform():
    if sys.platform.startswith('linux'):
        return 'linux'
    elif sys.platform.startswith('win32'):
        return 'win32'
    elif sys.platform.startswith('cygwin'):
        return 'cygwin'
    elif sys.platform.startswith('darwin'):
        return 'darwin'
    else:
        return sys.platform


def get_mac_version(run_lambda):
    return run_and_parse_first_match(run_lambda, 'sw_vers -productVersion', r'(.*)')


def get_windows_version(run_lambda):
    return run_and_read_all(run_lambda, 'wmic os get Caption | findstr /v Caption')


def get_lsb_version(run_lambda):
    return run_and_parse_first_match(run_lambda, 'lsb_release -a', r'Description:\t(.*)')


def check_release_file(run_lambda):
    return run_and_parse_first_match(run_lambda, 'cat /etc/*-release',
                                     r'PRETTY_NAME="(.*)"')


def get_os(run_lambda):
    platform = get_platform()

    if platform is 'win32' or platform is 'cygwin':
        return get_windows_version(run_lambda)

    if platform == 'darwin':
        version = get_mac_version(run_lambda)
        if version is None:
            return None
        return 'Mac OSX {}'.format(version)

    if platform == 'linux':
        # Ubuntu/Debian based
        desc = get_lsb_version(run_lambda)
        if desc is not None:
            return desc

        # Try reading /etc/*-release
        desc = check_release_file(run_lambda)
        if desc is not None:
            return desc

        return platform

    # Unknown platform
    return platform


def get_pip_packages(run_lambda):
    # People generally have `pip` as `pip` or `pip3`
    def run_with_pip(pip):
        if get_platform() == 'win32':
            grep_cmd = r'findstr /R "numpy torch"'
        else:
            grep_cmd = r'grep "torch\|numpy"'
        return run_and_read_all(run_lambda, pip + ' list --format=legacy | ' + grep_cmd)

    if not PY3:
        return 'pip', run_with_pip('pip')

    # Try to figure out if the user is running pip or pip3.
    out2 = run_with_pip('pip')
    out3 = run_with_pip('pip3')

    num_pips = len([x for x in [out2, out3] if x is not None])
    if num_pips is 0:
        return 'pip', out2

    if num_pips == 1:
        if out2 is not None:
            return 'pip', out2
        return 'pip3', out3

    # num_pips is 2. Return pip3 by default b/c that most likely
    # is the one associated with Python 3
    return 'pip3', out3


def get_env_info():
    run_lambda = run
    pip_version, pip_list_output = get_pip_packages(run_lambda)

    if TORCH_AVAILABLE:
        version_str = torch.__version__
        debug_mode_str = torch.version.debug
        cuda_available_str = torch.cuda.is_available()
        cuda_version_str = torch.version.cuda
    else:
        version_str = debug_mode_str = cuda_available_str = cuda_version_str = 'N/A'

    return SystemEnv(
        torch_version=version_str,
        is_debug_build=debug_mode_str,
        python_version='{}.{}'.format(sys.version_info[0], sys.version_info[1]),
        is_cuda_available=cuda_available_str,
        cuda_compiled_version=cuda_version_str,
        cuda_runtime_version=get_running_cuda_version(run_lambda),
        nvidia_gpu_models=get_gpu_info(run_lambda),
        nvidia_driver_version=get_nvidia_driver_version(run_lambda),
        cudnn_version=get_cudnn_version(run_lambda),
        pip_version=pip_version,
        pip_packages=pip_list_output,
        conda_packages=get_conda_packages(run_lambda),
        os=get_os(run_lambda),
        gcc_version=get_gcc_version(run_lambda),
        cmake_version=get_cmake_version(run_lambda),
    )

env_info_fmt = """
PyTorch version: {torch_version}
Is debug build: {is_debug_build}
CUDA used to build PyTorch: {cuda_compiled_version}
OS: {os}
GCC version: {gcc_version}
CMake version: {cmake_version}
Python version: {python_version}
Is CUDA available: {is_cuda_available}
CUDA runtime version: {cuda_runtime_version}
GPU models and configuration: {nvidia_gpu_models}
Nvidia driver version: {nvidia_driver_version}
cuDNN version: {cudnn_version}
Versions of relevant libraries:
{pip_packages}
{conda_packages}
""".strip()


def pretty_str(envinfo):
    def replace_nones(dct, replacement='Could not collect'):
        for key in dct.keys():
            if dct[key] is not None:
                continue
            dct[key] = replacement
        return dct

    def replace_bools(dct, true='Yes', false='No'):
        for key in dct.keys():
            if dct[key] is True:
                dct[key] = true
            elif dct[key] is False:
                dct[key] = false
        return dct

    def prepend(text, tag='[prepend]'):
        lines = text.split('\n')
        updated_lines = [tag + line for line in lines]
        return '\n'.join(updated_lines)

    def replace_if_empty(text, replacement='No relevant packages'):
        if text is not None and len(text) == 0:
            return replacement
        return text

    def maybe_start_on_next_line(string):
        # If `string` is multiline, prepend a \n to it.
        if string is not None and len(string.split('\n')) > 1:
            return '\n{}\n'.format(string)
        return string

    mutable_dict = envinfo._asdict()

    # If nvidia_gpu_models is multiline, start on the next line
    mutable_dict['nvidia_gpu_models'] = \
        maybe_start_on_next_line(envinfo.nvidia_gpu_models)

    # If the machine doesn't have CUDA, report some fields as 'No CUDA'
    dynamic_cuda_fields = [
        'cuda_runtime_version',
        'nvidia_gpu_models',
        'nvidia_driver_version',
    ]
    all_cuda_fields = dynamic_cuda_fields + ['cudnn_version']
    all_dynamic_cuda_fields_missing = all(
        mutable_dict[field] is None for field in dynamic_cuda_fields)
    if TORCH_AVAILABLE and not torch.cuda.is_available() and all_dynamic_cuda_fields_missing:
        for field in all_cuda_fields:
            mutable_dict[field] = 'No CUDA'
        if envinfo.cuda_compiled_version is None:
            mutable_dict['cuda_compiled_version'] = 'None'

    # Replace True with Yes, False with No
    mutable_dict = replace_bools(mutable_dict)

    # Replace all None objects with 'Could not collect'
    mutable_dict = replace_nones(mutable_dict)

    # If either of these are '', replace with 'No relevant packages'
    mutable_dict['pip_packages'] = replace_if_empty(mutable_dict['pip_packages'])
    mutable_dict['conda_packages'] = replace_if_empty(mutable_dict['conda_packages'])

    # Tag conda and pip packages with a prefix
    # If they were previously None, they'll show up as ie '[conda] Could not collect'
    if mutable_dict['pip_packages']:
        mutable_dict['pip_packages'] = prepend(mutable_dict['pip_packages'],
                                               '[{}] '.format(envinfo.pip_version))
    if mutable_dict['conda_packages']:
        mutable_dict['conda_packages'] = prepend(mutable_dict['conda_packages'],
                                                 '[conda] ')
    return env_info_fmt.format(**mutable_dict)


def get_pretty_env_info():
    return pretty_str(get_env_info())


def main():
    print("Collecting environment information...")
    output = get_pretty_env_info()
    print(output)


if __name__ == '__main__':
    main()

We can also use this code which seems simpler. You might need to run is separate from jupyter notebook though.
https://gist.github.com/f0k/63a664160d016a491b2cbea15913d549
    

In [ ]:
# Covariance matrix heatmap visualization as a sanity check

if DEBUG >=4:

    import seaborn as sns

    plt.figure(figsize=(10,10))
    sns.set(font_scale=1.5)

    # We need as many as cols as we have features
    cols=['1','2','3','4','5','6']

    # visualization for cluster number
    cluster_number = 2
    hm = sns.heatmap(gmm_covariances[cluster_number,:,:],
                     cbar=True,
                     annot=True,
                     square=True,
                     fmt='.5f',
                     annot_kws={'size': 12},
                     yticklabels=cols,
                     xticklabels=cols)

    plt.title('Covariance matrix of the cluster showing correlation coefficients')
    plt.tight_layout()
    plt.show()

In [ ]:
if DEBUG >=4:
    if (h2o4gpu_enabled == True):
        from h2o4gpu import metrics
    else:
        from sklearn import metrics

    metrics.silhouette_score(data_scaled, gmm_hash_clusters)

In [ ]:
#Read the data files and concantenate into a dataframe
import pandas as pd
import glob
import os

training_data_files_path = r'/home/kc/junk/Training_data_from_public_git/'                     
all_files = glob.glob(os.path.join(training_data_files_path, "*.csv"))     

for file in all_files:
    print('Processing: ',file)
    commits = pd.read_csv(file)
    commits['committed_date'] = \
        commits['committed_date'].astype('str').apply(lambda x: pd.to_datetime(x).tz_convert('GMT'))
    commits['authored_date'] = \
        commits['authored_date'].astype('str').apply(lambda x: pd.to_datetime(x).tz_convert('GMT'))
    commits.to_csv(file)

In [ ]:
#Read the data files
import pandas as pd

training_data_files_path = r'/home/kc/Projects/data_files/Training_data_from_public_git/'                     

angular_commits = pd.read_csv(training_data_files_path+'angularjs.git.csv')
#vscode_commits=pd.read_csv('/home/kc/Projects/data_files/vscode.csv')
react_commits=pd.read_csv(training_data_files_path+'react-native.git.csv')

#total_commits=tensorflow_commits.append(vscode_commits, ignore_index=True)
total_commits=angular_commits.append(react_commits, ignore_index=True)